In [1]:
import numpy as np
import pandas as pd
import sys, os
from random import shuffle
from tqdm import tqdm
import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx
import torch
import torch.nn as nn
from models.gat import GATNet
from models.gat_gcn import GAT_GCN
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import *
from os import walk


In [2]:
def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in tqdm(loader):
            data = data.to(device)
            output = model(data)
            total_preds = torch.cat((total_preds.to(device), output), 0)
    return total_preds.cpu().numpy().flatten()

In [3]:
benzos_path = 'prediction_for_generated_benzos_all/'
vdr_path = 'prediction_for_generated_vitamind/'

In [4]:
D_benzos = {}
D_vitamind = {}

benzos_path = "Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/csv_format/"
for file in os.listdir(benzos_path):
    if '.csv' in file:
        df = pd.read_csv(benzos_path+file)
        name = file.replace('.csv','')
        D_benzos[name] = df
        
        
        
vdr_path = "VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/"
for file in os.listdir(vdr_path):
    if '.csv' in file:
        df = pd.read_csv(vdr_path+file)
        name = file.replace('.csv','')
        D_vitamind[name] = df

In [5]:
md1 = '../GraphDTA_Results/Davis/GAT_GCN_Davis/model_GAT_GCN_davis.model'
md2 = '../GraphDTA_Results/Davis/GATNet_Davis/model_GATNet_davis.model'
md3 = '../GraphDTA_Results/Davis/GATNet_Davis/model_GATNet_davis2.model'
md4 = '../GraphDTA_Results/Davis/GCNNet_Davis/model_GCNNet_davis.model'
md5 = '../GraphDTA_Results/Davis/GINConvNet_Davis/model_GINConvNet_davis.model'


md6 = '../GraphDTA_Results/KIBA/GAT_GCN_KIBA/model_GAT_GCN_kiba.model'
md7 = '../GraphDTA_Results/KIBA/GATNet_KIBA/model_GATNet_kiba.model'
md8 = '../GraphDTA_Results/KIBA/GCNNet_KIBA/model_GCNNet_kiba.model'
md9 = '../GraphDTA_Results/KIBA/GINConvNet_KIBA/model_GINConvNet_kiba.model'

md10 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_Kd.model'
md11 = '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_bdtdc_ic50.model'
md12 = '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_IC50.model'
md13 = '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_Kd.model'

md14 = '../GraphDTA_Results/BindingDB/GCNNet/bdtdc_ic50_dec20/model_GCNNet_bdtdc_ic50.model'
md15 = '../GraphDTA_Results/BindingDB/GCNNet/strange_model_try_to_test/model_GCNNet_Kd.model'
md16 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bdtdc_ic50.model'
md17 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bdtdc_kd.model'

md18 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bdtdc_ki.model'
md19 = '../GraphDTA_Results/BindingDB/GINConvNet/model_GINConvNet_bindingdb_Kd.model'
md20 = '../GraphDTA_Results/BindingDB/GINConvNet/model_GINConvNet_IC50.model'
md21 = '../GraphDTA_Results/BindingDB/GINConvNet/model_GINConvNet_Ki.model'

md22 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_bdtdc_ki.model'
md23 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_bdtdc_ic50.model'

md24 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bindingdb_kd.model'
md25 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_bindingdb_kd.model'

md26 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bindingdb_ki.model'
md27 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_bindingdb_ki.model'

md28 = '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_bindingdb_ic50.model'
md29 = '../GraphDTA_Results/BindingDB/GCNNet/model_GCNNet_bindingdb_ic50.model'

md30 = '../GraphDTA_Results/BindingDB/bd_tdc_latest_models/model_GCNNet_bdtdc_ic50.model'
md31 = '../GraphDTA_Results/BindingDB/bd_tdc_latest_models/model_GCNNet_bdtdc_ki.model'

In [6]:
models_list = [md1, md2, md3, md4,md5, md6, md7, md8,md9, md10, md11, md12,md13, md14, md15, md16, md17, md18, md19, md20,md21, md22,md23, md24,md25,md26,md27,md28,md29,md30,md31 ]
model_names = [md1[46:-6], md2[45:-6],'GATNet_davis2', 'GCNNet_davis', 'GINConvNet_davis', 'GAT_GCN_kiba', 'GATNet_kiba', 'GCNNet_kiba', 'GINConvNet_kiba', 'GAT_GCN_Kd', 'GATNet_bdtdc_ic50', 'GATNet_IC50', 'GATNet_Kd', 'bdtdc_ic50_dec20/model_GCNNet_bdtdc_ic50','strange_model_try_to_test/model_GCNNet_Kd', 'GCNNet_bdtdc_ic50', 'GCNNet_bdtdc_kd', 'GCNNet_bdtdc_ki', 'GINConvNet_bindingdb_Kd', 'GINConvNet_IC50', 'GINConvNet_Ki', 'GAT_GCN_bdtdc_ki_latest', 'GAT_GCN_bdtdc_ic50_latest', 'GCNNet_bindingdb_kd_latest', 'GAT_GCN_bindingdb_kd_latest', 'GCNNet_bindingdb_ki_latest', 'GAT_GCN_bindingdb_ki_latest', 'GAT_GCN_bindingdb_ic50_latest', 'GCNNet_bindingdb_ic50_latest','GCNNet_bdtdc_ic50_latest_v2', 'GCNNet_bdtdc_ki_latest_v2']
model_arch = [GAT_GCN, GATNet,GATNet, GCNNet, GINConvNet, GAT_GCN, GATNet, GCNNet, GINConvNet, GAT_GCN, GATNet, GATNet, GATNet, GCNNet,GCNNet, GCNNet, GCNNet, GCNNet, GINConvNet, GINConvNet, GINConvNet,GAT_GCN,GAT_GCN, GCNNet,GAT_GCN,GCNNet,GAT_GCN,GAT_GCN, GCNNet,GCNNet,GCNNet]

In [7]:
models_list

['../GraphDTA_Results/Davis/GAT_GCN_Davis/model_GAT_GCN_davis.model',
 '../GraphDTA_Results/Davis/GATNet_Davis/model_GATNet_davis.model',
 '../GraphDTA_Results/Davis/GATNet_Davis/model_GATNet_davis2.model',
 '../GraphDTA_Results/Davis/GCNNet_Davis/model_GCNNet_davis.model',
 '../GraphDTA_Results/Davis/GINConvNet_Davis/model_GINConvNet_davis.model',
 '../GraphDTA_Results/KIBA/GAT_GCN_KIBA/model_GAT_GCN_kiba.model',
 '../GraphDTA_Results/KIBA/GATNet_KIBA/model_GATNet_kiba.model',
 '../GraphDTA_Results/KIBA/GCNNet_KIBA/model_GCNNet_kiba.model',
 '../GraphDTA_Results/KIBA/GINConvNet_KIBA/model_GINConvNet_kiba.model',
 '../GraphDTA_Results/BindingDB/GAT_GCN/model_GAT_GCN_Kd.model',
 '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_bdtdc_ic50.model',
 '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_IC50.model',
 '../GraphDTA_Results/BindingDB/GATNet/model_GATNet_Kd.model',
 '../GraphDTA_Results/BindingDB/GCNNet/bdtdc_ic50_dec20/model_GCNNet_bdtdc_ic50.model',
 '../GraphDTA_Results/Bindi

In [8]:
for i in range(len(models_list)):
    device = torch.device('cuda:4' if torch.cuda.is_available() else "cpu")
    modeling = model_arch[i]
    model = modeling()
    model.to(device)
    model.load_state_dict(torch.load(models_list[i],map_location='cuda:4'))

    TEST_BATCH_SIZE = 512
    for k,v in D_benzos.items():
        test_data = TestbedDataset(root=benzos_path, dataset=str(k))
        test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
        pred = predicting(model, device, test_loader)
        D_benzos[k][model_names[i]] = pred

    for k,v in D_vitamind.items():
        test_data = TestbedDataset(root=vdr_path, dataset=str(k))
        test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
        pred = predicting(model, device, test_loader)
        D_vitamind[k][model_names[i]] = pred


Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/csv_format/processed/LOWDATAREGIME_molecules_30_0.7_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 395 samples...


100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/csv_format/processed/TRANSVAE_benzos_pubchem_top_dims_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 1001 samples...


100%|██████████| 2/2 [00:00<00:00,  2.29it/s]


Pre-processed data found: Benzos-structures-filteres-by-similarity-to-diazepam-20211216T105015Z-001/csv_format/processed/LOWDATAREGIME_molecules_20_0.7_BENZOS_TOP_SIMILAR.pt, loading ...
Make prediction for 475 samples...


...

100%|██████████| 2/2 [00:00<00:00,  9.00it/s]


Pre-processed data found: VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/processed/TRANSMOL_VDR_TOP_SIMILAR.pt, loading ...
Make prediction for 1001 samples...


100%|██████████| 2/2 [00:00<00:00,  8.88it/s]


Pre-processed data found: VitaminDZinc-structures-filteres-by-similarity-to-calcitriol/csv_format/processed/LOWDATAREGIME_molecules_10_0.7_VDR_TOP_SIMILAR.pt, loading ...
Make prediction for 793 samples...


100%|██████████| 2/2 [00:00<00:00, 10.39it/s]


In [9]:
D_vitamind['LOWDATAREGIME_molecules_10_0.7_VDR_TOP_SIMILAR']

,Drug,Target,Y,GAT_GCN_davis,GATNet_davis,GATNet_davis2,GCNNet_davis,GINConvNet_davis,GAT_GCN_kiba,GATNet_kiba,...,GAT_GCN_bdtdc_ki_latest,GAT_GCN_bdtdc_ic50_latest,GCNNet_bindingdb_kd_latest,GAT_GCN_bindingdb_kd_latest,GCNNet_bindingdb_ki_latest,GAT_GCN_bindingdb_ki_latest,GAT_GCN_bindingdb_ic50_latest,GCNNet_bindingdb_ic50_latest,GCNNet_bdtdc_ic50_latest_v2,GCNNet_bdtdc_ki_latest_v2
0,CC(CCCC(C)(C)O)C1CCC2C1(CCCC2=CC=C3CC(CC(C3=C)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,6.163424,5.611393,5.386049,5.143214,5.143952,10.954560,10.292741,...,8.341628,4.642188,9.265543,6.598283,7.592827,8.693158,7.631602,6.123209,7.292428,5.926140
1,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)C(C)CCCC(C)(C)O)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,6.182073,5.736390,5.383852,5.282595,5.130151,10.875734,10.415213,...,8.519048,6.084364,9.440131,8.118802,7.079644,8.041803,7.952172,6.218816,7.308613,6.220350
2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(CC)CCCC(C)(C)O)CC(...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,5.985836,5.515864,5.253991,5.147671,5.089901,11.087587,10.260958,...,8.881292,4.530238,9.298445,6.773960,7.539011,8.502220,7.727466,6.294739,7.618228,5.822350
3,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(C)(C)O)CC(O...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,6.163424,5.611393,5.386049,5.143214,5.143952,10.954560,10.292741,...,8.341316,4.642188,9.265543,6.598283,7.592827,8.693158,7.631602,6.123209,7.292428,5.926140
4,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCCC(C)(C)O)CC(...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,6.152519,5.612009,5.384963,5.121865,5.147013,10.957424,10.321073,...,8.963316,4.858073,9.280178,6.278106,7.478539,8.501718,7.649857,6.040468,7.268254,6.546758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,CCC(CC)(c1ccc(C(CC)(CC)c2ccc(OC(C)CO)c(C)c2)cc...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,5.889183,6.536324,5.819087,5.763204,5.056121,11.946462,11.442703,...,6.596735,4.486119,5.123273,6.528322,6.262189,6.275127,4.595201,5.026949,4.628216,6.707725
789,CCC(C)(O)c1ccc(C(CC)(CC)c2ccc(C(O)(CC)CC)cc2)cc1C,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,6.115541,6.380451,5.524305,5.647188,5.118958,11.820581,11.696827,...,6.651388,5.000602,4.972356,5.629647,5.566422,6.340684,4.133740,5.099666,4.638898,6.497919
790,CCC(CC)(c1ccccc1)c1ccc(OCC(O)C(C)(C)C)cc1,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,6.051188,6.263648,5.616045,5.247438,5.312230,11.586207,11.590782,...,6.491522,4.556950,6.032476,7.311465,6.756710,7.247196,4.581574,4.823371,4.841097,6.771056
791,CCC(O)(CC)c1ccc(C(C)(CC)c2ccc(OCCCO)cc2)cc1,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,0.0,6.042235,5.544349,5.142534,5.486377,5.333824,11.595806,11.223198,...,7.403069,4.523956,5.197701,5.720027,5.934620,7.691916,4.618683,4.690276,4.587601,5.801325


In [11]:
for k,v in D_benzos.items():
    D_benzos[k].to_csv('prediction_for_generated_benzos_latest/GraphDTA_prediction_'+str(k)+'.csv',index= False )

for k,v in D_vitamind.items():
    D_vitamind[k].to_csv('prediction_for_generated_vitamind_latest/GraphDTA_prediction_'+str(k)+'.csv',index= False )


In [24]:
dfs_vdr[0]

,Drug,Target,GAT_GCN_davis,GATNet_davis,GATNet_davis2,GCNNet_davis,GINConvNet_davis,GAT_GCN_kiba,GATNet_kiba,GCNNet_kiba,...,GINConvNet_IC50,GINConvNet_Ki,GAT_GCN_bdtdc_ki_latest,GAT_GCN_bdtdc_ic50_latest,GCNNet_bindingdb_kd_latest,GAT_GCN_bindingdb_kd_latest,GCNNet_bindingdb_ki_latest,GAT_GCN_bindingdb_ki_latest,GAT_GCN_bindingdb_ic50_latest,GCNNet_bindingdb_ic50_latest
0,C=C1C(=CC=C2CC(C)CC3(C)C2CCC3C(C)C(C)CCC(C)(C)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,6.203391,5.694902,6.032289,5.572998,5.203593,10.537890,10.613931,10.429836,...,5.384230,8.825441,8.470876,7.080710,9.259553,7.125468,6.659076,8.467819,7.860690,4.980411
1,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)(C)C(C)CCC(C)(C)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,6.194114,5.684735,5.317656,5.238730,5.213115,11.077236,10.531986,10.518936,...,7.475556,8.837432,8.419527,5.757961,9.882589,6.889663,7.254718,8.489131,6.798367,6.265611
2,C=C1C(=CC=C2CCCC3(CCCC)C2CCC3C(C)C(C)CCC(C)(C)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,6.428329,5.611572,5.242209,5.495798,5.284415,10.824659,10.593780,10.393126,...,5.939070,8.652788,8.719942,4.742641,9.905434,8.431267,7.832273,8.692762,7.617512,6.572859
3,C=C1C(=CC=C2CCCC3(C)C2CCC3C(CC)C(C)CCC(C)(C)O)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,6.317272,5.586719,5.307574,5.183216,5.152680,11.002387,10.350780,10.559709,...,6.334094,8.524074,8.529881,4.823941,9.777245,8.658220,7.336759,8.470592,7.697564,6.208617
4,C=C1C(=CC=C2CCCC3(C)C2CCC3C(CCCC)C(C)CCC(C)(C)...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,6.362350,5.564388,5.296394,5.262090,5.128633,10.924446,10.554974,10.395580,...,6.351528,8.420326,8.433319,5.472513,9.801883,7.217220,7.879875,8.609930,7.693279,7.154109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,CCC(CC)(c1ccccc1)c1ccc(OCC(=O)C(C)(C)C)c(C)c1,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,5.836588,6.330083,5.814915,5.488825,5.496130,11.619188,11.392080,11.011942,...,6.705483,6.633827,6.047816,4.527653,5.260979,5.299569,6.269884,6.107968,4.566155,4.635391
996,CCC(CC)(c1cccc(C)c1)c1ccc(OCC(=O)C(C)(C)C)c(C)c1,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,5.826588,6.327671,5.711961,5.191518,5.514371,11.674183,11.373125,11.161098,...,6.648481,6.496977,6.177055,4.693862,5.079478,5.727530,6.361308,6.111541,4.635936,4.695464
997,CCC(CC)(c1ccc(O)c(C)c1)c1ccc(OCC(=O)C(C)(C)C)c...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,6.072472,6.296263,5.403594,5.051076,5.231698,11.920741,11.356113,12.165985,...,6.564181,6.656836,6.332341,4.287520,5.455119,5.739219,6.572809,6.203876,4.398159,4.633916
998,CCCOc1ccc(C(CC)(CC)c2ccc(OCC(=O)C(C)(C)C)c(C)c...,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,5.887292,6.220612,5.703904,5.202060,5.244125,11.602036,11.321152,11.167489,...,6.312003,6.457648,6.151821,5.218818,5.084465,5.375141,6.292008,6.122578,4.898650,4.628778


In [26]:
filenames_benzos

['GraphDTA_prediction_LOWDATAREGIME_molecules_10_0.7_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_TRANSVAE_benzos_zinc_top_dims_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_LOWDATAREGIME_molecules_40_0.7_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_LOWDATAREGIME_molecules_20_0.7_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_TRANSVAE_benzos_pubchem_top_dims_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_TRANSVAE_benzos_zinc_k_dims_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_TRANSMOL_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_LOWDATAREGIME_molecules_30_0.7_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_TRANSVAE_benzos_pubchem_k_dims_BENZOS_TOP_SIMILAR.csv',
 'GraphDTA_prediction_REINVENT_BENZOS_TOP_SIMILAR.csv']

In [31]:
filenames_vdr[0]

'GraphDTA_prediction_TRANSMOL_VDR_TOP_SIMILAR.csv'

In [30]:
dfs_benzos[0]

,Drug,Target,GAT_GCN_davis,GATNet_davis,GATNet_davis2,GCNNet_davis,GINConvNet_davis,GAT_GCN_kiba,GATNet_kiba,GCNNet_kiba,...,GINConvNet_IC50,GINConvNet_Ki,GAT_GCN_bdtdc_ki_latest,GAT_GCN_bdtdc_ic50_latest,GCNNet_bindingdb_kd_latest,GAT_GCN_bindingdb_kd_latest,GCNNet_bindingdb_ki_latest,GAT_GCN_bindingdb_ki_latest,GAT_GCN_bindingdb_ic50_latest,GCNNet_bindingdb_ic50_latest
0,CN1C(=O)CN=C(c2ccccc2F)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.764262,5.693633,5.844633,5.790274,5.317926,11.537140,11.585246,10.905057,...,6.530537,8.019577,6.781021,4.908838,7.518653,7.580321,8.552409,7.917223,4.408779,4.199214
1,O=C1CN=C(c2ccccc2)c2cc(Cl)ccc2N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.820510,5.347934,5.088555,5.706682,5.599361,11.650953,11.413750,10.463663,...,4.286624,6.693745,7.465182,5.336170,8.336432,6.683643,6.932455,6.525292,4.077450,4.242430
2,CN1CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.781652,6.222953,5.784204,5.979016,5.766552,11.689509,11.376568,10.623102,...,5.920058,7.680098,7.867999,5.522687,7.871615,8.290163,7.346959,7.061687,4.753488,3.327381
3,CN1C(=O)CN=C(c2ccccc2)c2cc(N)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.703455,5.775535,5.232235,5.181041,5.449607,11.536312,11.222670,11.156598,...,5.791874,7.391438,6.965907,4.801177,7.801600,8.492088,7.365209,7.284909,4.722075,4.560641
4,CN1C(=O)CCN=C(c2ccccc2)c2cc(Cl)ccc21,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.347439,6.370338,5.859731,5.626638,5.456227,11.525576,11.430586,10.467786,...,5.418643,7.303138,7.689903,5.136808,7.275914,7.490314,7.564006,6.763002,4.466407,3.770662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,NC1=NC(C=Cc2ccc3nccnc3c2)CCC1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.988225,6.207549,5.812002,5.292656,5.809867,11.671762,11.476151,10.881434,...,5.277848,8.199919,6.887542,6.513785,9.489430,9.410442,7.423059,7.214113,5.900526,5.504848
484,O=C(OC1CC1)c1nccc2cc(-c3ccccc3)ccc12,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,6.026775,5.510842,5.401698,5.795946,5.684030,11.552782,11.378396,10.424806,...,6.865233,7.098046,6.925295,5.284789,8.850294,9.850992,6.320870,6.261251,4.025298,5.787142
485,O=C1NCC1=COc1ccc2cncnc2c1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.160220,5.217212,5.890978,5.327335,5.549436,11.563120,11.498323,10.587861,...,6.189260,8.255099,6.086030,6.323635,8.091886,6.373436,6.761993,6.778954,5.007416,4.465621
486,COC1CN=c2ccc(Cl)cc2=N1,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,5.912923,6.358792,5.317985,5.248207,5.167810,11.467406,11.234021,10.676084,...,4.699046,6.586902,7.038149,5.808917,8.711000,6.419726,6.162477,5.837805,4.480692,4.271014
